<a href="https://colab.research.google.com/github/mmigo1/task_pandas/blob/main/trial_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Найти тариф стоимости доставки для каждого склада

In [ ]:
import pandas as pd
import json
import requests

try:
  with open('trial_task.json','r',encoding='utf-8') as f:
      file  = json.loads(f.read())
except:
  resp = requests.get('https://file.notion.so/f/s/0f8850ad-e46f-4f37-99ea-0e4e2a6af5b6/trial_task.json?id=2583a04b-4256-4c1f-939e-6eac0f749ceb&table=block&spaceId=41165294-a784-489a-a401-1a916d020564&expirationTimestamp=1690192800000&signature=SPxrcEG3lFTCC0IMyayetbnAAHuYBmOojBHytHMKm64&downloadName=trial_task.json')
  file = resp.json()

df = pd.json_normalize(file,record_path='products',meta=['order_id','warehouse_name','highway_cost'])
df['quantity_all'] = df.groupby('order_id')['quantity'].transform('sum')
df['tariff_cost'] = abs(df['highway_cost'] / df ['quantity_all'])
df['tariff_cost'] = df['tariff_cost'].astype('int32')
df

,product,price,quantity,order_id,warehouse_name,highway_cost,quantity_all,tariff_cost
0,ломтик июльского неба,450,1,11973,Мордор,-70,7,10
1,билет в Израиль,1000,3,11973,Мордор,-70,7,10
2,статуэтка Ленина,200,3,11973,Мордор,-70,7,10
3,билет в Израиль,1000,1,62239,хутор близ Диканьки,-15,1,15
4,зеленая пластинка,10,2,85794,отель Лето,-50,2,25
...,...,...,...,...,...,...,...,...
192,автограф Стаса Барецкого,600,1,79293,отель Лето,-75,3,25
193,ломтик июльского неба,450,1,79293,отель Лето,-75,3,25
194,плюмбус,250,2,2930,Мордор,-30,3,10
195,плюмбус,250,1,2930,Мордор,-30,3,10


In [ ]:
df.groupby(['warehouse_name','order_id']).agg({
    'tariff_cost':'sum',
})

tariff_cost
warehouse_name      order_id             
Мордор              2930               20
                    11973              30
                    16240              10
                    26702              20
                    27443              10
...                                   ...
хутор близ Диканьки 84471              45
                    85787              45
                    98423              15
                    99220              45
                    99246              15

[100 rows x 1 columns]

# Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами
#'product', 'quantity', 'income', 'expenses', 'profit')

In [ ]:
result = pd.DataFrame()
income = df.groupby('product').apply(lambda x: (x['price'] * x['quantity_all']).sum()).reset_index(name='income')
expenses = df.groupby('product').apply(lambda x: (x['tariff_cost'] * x['quantity_all']).sum()).reset_index(name='expenses')
result['product'] = df['product'].unique()
result['quantity'] = df.groupby('product')['quantity_all'].transform('sum')
result = result.merge(income)
result = result.merge(expenses)
result['profit'] = result['income'] - result['expenses']
result

,product,quantity,income,expenses,profit
0,ломтик июльского неба,80,36000,1210,34790
1,билет в Израиль,124,124000,2305,121695
2,статуэтка Ленина,158,31600,2235,29365
3,зеленая пластинка,124,1460,2260,-800
4,автограф Стаса Барецкого,146,72600,2090,70510
5,плюмбус,124,39750,2255,37495
6,подписка на suppi-блог,146,12900,1415,11485


#Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов

In [ ]:
result_order = pd.DataFrame()
income = df.groupby('order_id').apply(lambda x: (x['price'] * x['quantity']).sum()).reset_index(name='income')
expenses = df.groupby('order_id').apply(lambda x: (x['tariff_cost'] * x['quantity']).sum()).reset_index(name='expenses')
result_order['order_id'] = df['order_id'].unique()
result_order = result_order.merge(income)
result_order = result_order.merge(expenses)
result_order['order_profit'] = result_order['income'] - result_order['expenses']
result_order['median'] = result_order['order_profit'].median()
result_order= result_order.drop(['income','expenses'],axis=1)
result_order

,order_id,order_profit,median
0,11973,3980,985.0
1,62239,985,985.0
2,85794,-30,985.0
3,33684,1980,985.0
4,25824,975,985.0
...,...,...,...
95,28106,320,985.0
96,98100,1590,985.0
97,79293,1175,985.0
98,2930,720,985.0


# Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)


In [ ]:
result_wh = pd.DataFrame()
result_wh = df.groupby(['warehouse_name', 'product','highway_cost','price'])['quantity'].sum().reset_index()
result_wh['profit'] = result_wh['quantity'] * result_wh['price'] - (-result_wh['highway_cost']) * result_wh['quantity']
result_wh = result_wh.drop(['highway_cost','price'],axis=1)
result_wh = result_wh.groupby(['warehouse_name','product']).agg({
    'quantity':'sum',
    'profit':'sum'
})
result_wh['profit_total'] = result_wh.groupby('warehouse_name')['profit'].transform('sum')
result_wh['percent_profit_product_of_warehouse']= result_wh['profit'] * 100 / result_wh['profit_total']
result_wh = result_wh.drop(['profit_total'],axis=1)
result_wh

quantity  profit  \
warehouse_name      product                                      
Мордор              автограф Стаса Барецкого         4    2130   
                    билет в Израиль                  9    8650   
                    зеленая пластинка               11    -300   
                    ломтик июльского неба            3    1240   
                    плюмбус                          6    1170   
                    подписка на suppi-блог           8     700   
                    статуэтка Ленина                10    1560   
гиперборея          автограф Стаса Барецкого        12    5900   
                    билет в Израиль                 21   19000   
                    зеленая пластинка               10    -900   
                    ломтик июльского неба           13    4730   
                    плюмбус                          9    1190   
                    подписка на suppi-блог          12     600   
                    статуэтка Ленина                 9     600   
остров невезения    автограф Стаса Барецкого         2    1180   
                    билет в Израиль                  3    2975   
                    зеленая пластинка                9    -125   
                    ломтик июльского неба           10    4230   
                    плюмбус                         14    3175   
                    статуэтка Ленина                18    3185   
отель Лето          автограф Стаса Барецкого         8    3975   
                    билет в Израиль                 15   13075   
                    зеленая пластинка               10    -950   
                    ломтик июльского неба            9    3475   
                    плюмбус                          9    1425   
                    подписка на suppi-блог           3     125   
                    статуэтка Ленина                10     825   
хутор близ Диканьки автограф Стаса Барецкого        22   11685   
                    билет в Израиль                 10    9190   
                    зеленая пластинка               21   -1710   
                    ломтик июльского неба            7    2730   
                    плюмбус                         27    4635   
                    подписка на suppi-блог          10     855   
                    статуэтка Ленина                21    2865   

                                              percent_profit_product_of_warehouse  
warehouse_name      product                                                        
Мордор              автограф Стаса Барецкого                            14.059406  
                    билет в Израиль                                     57.095710  
                    зеленая пластинка                                   -1.980198  
                    ломтик июльского неба                                8.184818  
                    плюмбус                                              7.722772  
                    подписка на suppi-блог                               4.620462  
                    статуэтка Ленина                                    10.297030  
гиперборея          автограф Стаса Барецкого                            18.958869  
                    билет в Израиль                                     61.053985  
                    зеленая пластинка                                   -2.892031  
                    ломтик июльского неба                               15.199229  
                    плюмбус                                              3.823907  
                    подписка на suppi-блог                               1.928021  
                    статуэтка Ленина                                     1.928021  
остров невезения    автограф Стаса Барецкого                             8.071135  
                    билет в Израиль                                     20.348837  
                    зеленая пластинка                                   -0.854993  
                    ломтик июльского неба                               2

# Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться
# 'accumulated_percent_profit_product_of_warehouse'. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.

In [ ]:
sorted = result_wh.sort_values(by=['warehouse_name','percent_profit_product_of_warehouse'], ascending=False)
sorted['accumulated_percent_profit_product_of_warehouse'] = sorted.groupby(['warehouse_name'])['percent_profit_product_of_warehouse'].cumsum()
sorted

quantity  profit  \
warehouse_name      product                                      
хутор близ Диканьки автограф Стаса Барецкого        22   11685   
                    билет в Израиль                 10    9190   
                    плюмбус                         27    4635   
                    статуэтка Ленина                21    2865   
                    ломтик июльского неба            7    2730   
                    подписка на suppi-блог          10     855   
                    зеленая пластинка               21   -1710   
отель Лето          билет в Израиль                 15   13075   
                    автограф Стаса Барецкого         8    3975   
                    ломтик июльского неба            9    3475   
                    плюмбус                          9    1425   
                    статуэтка Ленина                10     825   
                    подписка на suppi-блог           3     125   
                    зеленая пластинка               10    -950   
остров невезения    ломтик июльского неба           10    4230   
                    статуэтка Ленина                18    3185   
                    плюмбус                         14    3175   
                    билет в Израиль                  3    2975   
                    автограф Стаса Барецкого         2    1180   
                    зеленая пластинка                9    -125   
гиперборея          билет в Израиль                 21   19000   
                    автограф Стаса Барецкого        12    5900   
                    ломтик июльского неба           13    4730   
                    плюмбус                          9    1190   
                    подписка на suppi-блог          12     600   
                    статуэтка Ленина                 9     600   
                    зеленая пластинка               10    -900   
Мордор              билет в Израиль                  9    8650   
                    автограф Стаса Барецкого         4    2130   
                    статуэтка Ленина                10    1560   
                    ломтик июльского неба            3    1240   
                    плюмбус                          6    1170   
                    подписка на suppi-блог           8     700   
                    зеленая пластинка               11    -300   

                                              percent_profit_product_of_warehouse  \
warehouse_name      product                                                         
хутор близ Диканьки автограф Стаса Барецкого                            38.628099   
                    билет в Израиль                                     30.380165   
                    плюмбус                                             15.322314   
                    статуэтка Ленина                                     9.471074   
                    ломтик июльского неба                                9.024793   
                    подписка на suppi-блог                               2.826446   
                    зеленая пластинка                                   -5.652893   
отель Лето          билет в Израиль                                     59.567198   
                    автограф Стаса Барецкого                            18.109339   
                    ломтик июльского неба                               15.831435   
                    плюмбус                                              6.492027   
                    статуэтка Ленина                                     3.758542   
                    подписка на suppi-блог                               0.569476   
                    зеленая пластинка                                   -4.328018   
остров невезения    ломтик июльского неба                               28.932969   
                    статуэтка Ленина                                    21.785226   
                    плюмбус                                             21.716826   
                    билет в Израиль                   

# Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'). Если значение накопленного процента меньше или равно 70, то категория A.
#Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как 'category'

In [ ]:
sorted['category'] = sorted['accumulated_percent_profit_product_of_warehouse'].apply(lambda x: 'B' if x > 70 and x <= 90 else 'A' if x <= 70 else 'C')
sorted

quantity  profit  \
warehouse_name      product                                      
хутор близ Диканьки автограф Стаса Барецкого        22   11685   
                    билет в Израиль                 10    9190   
                    плюмбус                         27    4635   
                    статуэтка Ленина                21    2865   
                    ломтик июльского неба            7    2730   
                    подписка на suppi-блог          10     855   
                    зеленая пластинка               21   -1710   
отель Лето          билет в Израиль                 15   13075   
                    автограф Стаса Барецкого         8    3975   
                    ломтик июльского неба            9    3475   
                    плюмбус                          9    1425   
                    статуэтка Ленина                10     825   
                    подписка на suppi-блог           3     125   
                    зеленая пластинка               10    -950   
остров невезения    ломтик июльского неба           10    4230   
                    статуэтка Ленина                18    3185   
                    плюмбус                         14    3175   
                    билет в Израиль                  3    2975   
                    автограф Стаса Барецкого         2    1180   
                    зеленая пластинка                9    -125   
гиперборея          билет в Израиль                 21   19000   
                    автограф Стаса Барецкого        12    5900   
                    ломтик июльского неба           13    4730   
                    плюмбус                          9    1190   
                    подписка на suppi-блог          12     600   
                    статуэтка Ленина                 9     600   
                    зеленая пластинка               10    -900   
Мордор              билет в Израиль                  9    8650   
                    автограф Стаса Барецкого         4    2130   
                    статуэтка Ленина                10    1560   
                    ломтик июльского неба            3    1240   
                    плюмбус                          6    1170   
                    подписка на suppi-блог           8     700   
                    зеленая пластинка               11    -300   

                                              percent_profit_product_of_warehouse  \
warehouse_name      product                                                         
хутор близ Диканьки автограф Стаса Барецкого                            38.628099   
                    билет в Израиль                                     30.380165   
                    плюмбус                                             15.322314   
                    статуэтка Ленина                                     9.471074   
                    ломтик июльского неба                                9.024793   
                    подписка на suppi-блог                               2.826446   
                    зеленая пластинка                                   -5.652893   
отель Лето          билет в Израиль                                     59.567198   
                    автограф Стаса Барецкого                            18.109339   
                    ломтик июльского неба                               15.831435   
                    плюмбус                                              6.492027   
                    статуэтка Ленина                                     3.758542   
                    подписка на suppi-блог                               0.569476   
                    зеленая пластинка                                   -4.328018   
остров невезения    ломтик июльского неба                               28.932969   
                    статуэтка Ленина                                    21.785226   
                    плюмбус                                             21.716826   
                    билет в Израиль                   